In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
!pip install iterative-stratification

import numpy as np 
import scipy as sp
import sklearn as sl
import pandas as pd
import librosa
import IPython
import matplotlib.pyplot as plt
from dataclasses import dataclass
from functools import partial
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from tqdm import tqdm

import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.keras.layers as tfkl
import tensorflow.data as tfd
import tensorflow_addons as tfa
import tensorflow_hub as hub

import os
from kaggle_datasets import KaggleDatasets

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
cfg = { 
    'Training' : False,
    'parse_params': {
        'sample_rate' : 48000,
        'cut' : 5.5,
        'cut_step': 5.0
    },
    'data_params': {
        'frame_length' : 2048,
        'frame_step' : 512,
        'mel_highest_freq' : 16000.0,
        'mel_lowest_freq' : 100.0,
        'mel_num_bins' : 256,
        'img_shape' : (256, 512),
        'patch_cut' : 5.5
    },
    'model_params': {
        'ml_model': tf.keras.applications.ResNet50, #tf.keras.applications.EfficientNetB3
        'batchsize_per_tpu': 16,
        'ml_model': {
            'fn' : tf.keras.applications.ResNet50,
            'params' : {'include_top': False, 'weights' : 'imagenet', 'pooling' : 'avg' },
            'top_drops' : 0.4,
            'top_size' : 256,
            'name' : 'JustSecondModel'  
        },
    }
}

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
GCS_DS_PATH = KaggleDatasets().get_gcs_path("rfcx-species-audio-detection")

@dataclass(frozen=True)  # Instances of this class are immutable.
class Params:
    class_n: int = 24
    sr: float = cfg['parse_params']['sample_rate'] # all wave's sample rate may be 48k 
    cut: float = cfg['parse_params']['cut']
    cut_step: float = cfg['parse_params']['cut_step']  
    training: bool = cfg['Training']
        
    @property
    def pfrm_len(self):
        return int(cfg['parse_params']['cut'] * self.sr)
    @property
    def pfrm_step(self):
        return int(cfg['parse_params']['cut_step'] * self.sr)  
    @property
    def step_cnt(self):
        return int(60.0 / self.cut_step)
        
    frm_len: int = cfg['data_params']['frame_length']
    frm_step: int = cfg['data_params']['frame_step']
    mel_frq_h: float = cfg['data_params']['mel_highest_freq']
    mel_frq_l: float = cfg['data_params']['mel_lowest_freq']
    mel_bins: int = cfg['data_params']['img_shape'][0]
    img_size_h: int = cfg['data_params']['img_shape'][0]
    img_size_w: int = cfg['data_params']['img_shape'][1]
    patch_cut: float = cfg['data_params']['patch_cut']
        
    @property
    def model_fn(self):
        model_fn = cfg['model_params']['ml_model']
        model_fn['params']['input_shape'] = (self.img_size_h, self.img_size_w, 3)
        return model_fn

    test_tfrec: str = GCS_DS_PATH + "/tfrecords/test"
        
    subm_file: str = "submission_unspecified.csv"
    
    @property
    def all_test_tfrec(self):
        return [os.path.join(self.test_tfrec,filename) for filename in tf.io.gfile.listdir(self.test_tfrec)]

    path_tp_model: str = '/kaggle/input/model-weights-part5/model_v3_e52_s2048_minseg05.h5'
    path_fp_model: str = ''
    
GPARAMS = Params()

# Data Model

In [ ]:
feature_description = {
    'recording_id': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'audio_wav': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'label_info': tf.io.FixedLenFeature([], tf.string, default_value=''),
}

parse_dtype = {
    'audio_wav': tf.float32,
    'recording_id': tf.string,
}

@tf.function
def parse_test_fun(example_proto, params):
    sample = tf.io.parse_single_example(example_proto, feature_description)
    wav, _ = tf.audio.decode_wav(sample['audio_wav'], desired_channels=1)
    seq_len = tf.cast(params.sr * 60.0, tf.int32)
    wav = tf.reshape(wav,[seq_len])
    def _cut_slices(i):
        r = tf.math.minimum(seq_len, i + params.pfrm_len)
        s = wav[i : r - 1]
        paddings = tf.convert_to_tensor([[0, params.pfrm_len - r + i]])    
        s = tf.pad(s,paddings)
        return {'audio_wav' : s, 'recording_id' : sample['recording_id']}
    irange = tf.range(params.step_cnt, dtype = tf.int32) * params.pfrm_step
    return tf.map_fn(_cut_slices, irange, dtype = parse_dtype) 

@tf.function
def convert_to_spect(sample, params):
    waveform = sample["audio_wav"]
    with tf.name_scope('log_mel_features'):
        fft_length = 2 ** int(np.ceil(np.log(params.frm_len) / np.log(2.0)))
        num_spectrogram_bins = fft_length // 2 + 1
        magnitude_spectrogram = tf.abs(tf.signal.stft(signals=waveform,frame_length=params.frm_len,
                                                        frame_step=params.frm_step,fft_length=fft_length))
        linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(num_mel_bins=params.mel_bins,
                                                                            num_spectrogram_bins=num_spectrogram_bins,
                                                                            sample_rate=params.sr,
                                                                            lower_edge_hertz=params.mel_frq_l,
                                                                            upper_edge_hertz=params.mel_frq_h)
        mel_spectrogram = tf.matmul(magnitude_spectrogram, linear_to_mel_weight_matrix)
        log_mel_spectrogram = tf.math.log(mel_spectrogram + 0.001)
        return {'x' : tf.transpose(log_mel_spectrogram,[0,2,1])[...,:params.img_size_w], 'y' : sample["recording_id"]}
    

@tf.function
def fit_output(sample, params):
    img = tf.image.per_image_standardization(tf.expand_dims(sample['x'],-1)) 
    img = tf.squeeze(img)
    img = (img - tf.math.reduce_min(img)) / (tf.math.reduce_max(img) - tf.math.reduce_min(img))
    return (img, sample["y"])

# Machine Learning Model

In [ ]:
def create_model(cnn_model, params):
    for layer in cnn_model.layers:
        layer.trainable = True
    input = tfkl.Input(shape = (params.img_size_h, params.img_size_w))
    x = tfkl.Reshape((params.img_size_h, params.img_size_w,1))(input)
    x = tfkl.Conv2D(filters = 3, kernel_size = 7, padding='same', activation = 'sigmoid')(x)
    x = cnn_model(x)
    x = tfkl.Dropout(0.4)(x)
    x = tfkl.Dense(1024, activation='relu')(x)
    x = tfkl.Dropout(0.4)(x)
    output = tfkl.Dense(params.class_n, activation = 'sigmoid')(x)
    model = tfk.Model(name='yamnet_frames', inputs=input,outputs=output)
    return model

def create_modelV2(cnn_model, params):   
    inputs = tfkl.Input(shape = (params.img_size_h, params.img_size_w))
    x = tfkl.Reshape((params.img_size_h, params.img_size_w,1))(inputs)
    x = tfkl.Conv2D(filters = 3, kernel_size = 7, padding='same', activation = 'sigmoid')(x)
    x = cnn_model(x)
    outputs = []
    for _ in range(params.class_n):
        t = tfkl.Dropout(0.4)(x, training = False)
        t = tfkl.Dense(params.model_fn['top_size'], activation='relu')(t)
        t = tfkl.Dropout(params.model_fn['top_drops'])(t, training = False)
        t = tfkl.Dense(1, activation = 'sigmoid')(t)
        outputs.append(t)
    outputs = tfkl.concatenate(outputs)
    model = tfk.Model(name=params.model_fn['name'], inputs=inputs,outputs=outputs)
    return model

def FetchModel(params, ver = 2):
    cnn_model = cnn_model = params.model_fn['fn'](**params.model_fn['params'])
    model = None
    if ver == 1:
        model = create_model(cnn_model,params)
    else:
        model = create_modelV2(cnn_model,params)
    model.load_weights(params.path_tp_model)
    return model

# Prediction

In [ ]:
def make_mix(epoch, version, ms = '1',shuffle = 2048):
    root = './'
    files = [os.path.join(root,file) for file in os.listdir(root) if file.endswith(".csv")]
    cols = ['s{}'.format(i) for i in range(24)]
    df = pd.read_csv(files[0])
    mm = None
    for i,file in enumerate(files):
        print(f"{i} -- {file}")
        dff = pd.read_csv(file)
        m = dff[cols].to_numpy()
        if i == 0:
            mm = m
        else:
            mm = np.maximum(mm,m)
    df[cols] = mm
    df.to_csv(f'./submission_mix_v{version}_e{epoch}_ms{ms}_s{shuffle}_f16_f18_f20.csv', index = False)

#---------------------------------------------------------------------------------------------------------------------------

def make_prediction(params):
    print(params)
    print("-------------------Begin------------------------------------")
    testdataset = (tfd.TFRecordDataset(params.all_test_tfrec)
                   .map(partial(parse_test_fun, params = params), num_parallel_calls=AUTOTUNE)
                   .map(partial(convert_to_spect, params = params), num_parallel_calls=AUTOTUNE)
                   .map(partial(fit_output, params = params), num_parallel_calls=AUTOTUNE))

    model = FetchModel(params)

    recids = []
    rows = []
    for data,recid in tqdm(testdataset):
        prob = model(data, training = False)
        prob = tf.reduce_max(prob, axis = 0)
        row = [recid[0].numpy().decode()] + list(prob.numpy())
        rows.append(row)
    cols = ['recording_id'] + ['s{}'.format(i) for i in range(24)]
    subdf = pd.DataFrame(rows, columns = cols)
    subdf.to_csv(params.subm_file,index=False)
    print("------------------End---------------------------------------")

In [ ]:
'''
path_tp_model = ['/kaggle/input/model-weights-v7-fpf02/model_v7_e44_s2048_ms05_msfp2_u256_tp0_pfp02.h5',               
                 '/kaggle/input/model-weights-v7-fpf02/model_v7_e48_s2048_ms05_msfp2_u256_tp0_pfp02.h5',               
                 '/kaggle/input/model-weights-v7-fpf02/model_v7_e52_s2048_ms05_msfp2_u256_tp0_pfp02.h5',                 
                 '/kaggle/input/model-weights-v7-fpf02/model_v7_e56_s2048_ms05_msfp2_u256_tp0_pfp02.h5',
                 '/kaggle/input/model-weights-v7-fpf02/model_v7_e60_s2048_ms05_msfp2_u256_tp0_pfp02.h5',                
                 '/kaggle/input/model-weights-v7-fpf02/model_v7_e64_s2048_ms05_msfp2_u256_tp0_pfp02.h5',                 
                 '/kaggle/input/model-weights-v7-fpf02/model_v7_e68_s2048_ms05_msfp2_u256_tp0_pfp02.h5',
                 '/kaggle/input/model-weights-v7-fpf02/modelv7_e72_i128_fp_s512_s2048_minseg05_u256_fpf02.h5',
                 '/kaggle/input/model-weights-v7-fpf02/model_v7_e68_s2048_ms05_msfp2_u256_tp0_pfp02.h5',
                 '/kaggle/input/model-weights-v7-fpf02/modelv7_e72_i128_fp_s512_s2048_minseg05_u256_fpf02.h5']
epochs = [44,48,52,56,60,64,68,72,68,72]
frqhv = [(16000.0,),(16000.0,),(16000.0,),(16000.0,),(16000.0,),(16000.0,),(16000.0,),(16000.0,),(18000.0,),(18000.0,)]
'''
path_tp_model = ['/kaggle/../input/model-weights-v7-fpf03/model_v7_e64_s2048_ms05_msfp2_u256_tp0_pfp03.h5',               
                 '/kaggle/../input/model-weights-v7-fpf03/model_v7_e68_s2048_ms05_msfp2_u256_tp0_pfp03.h5',               
                 '/kaggle/../input/model-weights-v7-fpf03/model_v7_e72_i128_fp_s512_s2048_minseg05_u256.h5']
epochs = [64,68,72]
frqhv = [(16000.0,),(16000.0,),(16000.0,)]

version = 7
shuffle = 2048
u = 256
ms = '05'
fpf = '02'

for tp_path, frqv, e in zip(path_tp_model,frqhv,epochs):
    for i in range(len(frqv)):
        subm_file = f"submission_v{version}_e{e}_s{shuffle}_ms{ms}_f{int(frqv[i])}_u{u}_fpf{fpf}.csv"
        params = Params(subm_file = subm_file, mel_frq_h = frqv[i], path_tp_model = tp_path)
        make_prediction(params) 
    
make_mix(epoch = '64___72_pfp03', version = version, ms = ms, shuffle = shuffle)

In [ ]:
for i in range(1000000):
    print("########################################################################################################")

In [ ]:
epoch = 48
version = 5
shuffle = 2048
ms = '05'
path_tp_model = '/kaggle/input/model-weights-part5/model_v3_e48_s2048_minseg05.h5'
frqhv = [16000.0,18000.0,20000.0, 24000.0]

for frq in frqhv:
    subm_file = f"submission_v{version}_e{epoch}_s{shuffle}_ms{ms}_f{int(frq)}.csv"
    params = Params(subm_file = subm_file, mel_frq_h = frq, path_tp_model = path_tp_model)
    make_prediction(params) 
    
make_mix(epoch = epoch, version = version, ms = ms, shuffle = shuffle)

In [ ]:
for i in range(100000):
    print("########################################################################################################")

In [ ]:
path_tp_model = '/kaggle/input/model-weights-part4/modelv3_e28_i64_tp_s512_s2048.h5'
subm_file = "submission_v3_e28_i64_tp_s512_s2048_f20000.csv"
mel_frq_h = 20000.0

GPARAMS = Params(path_tp_model = path_tp_model, subm_file = subm_file, mel_frq_h = mel_frq_h)

In [ ]:
def make_prediction(params):
    testdataset = (tfd.TFRecordDataset(params.all_test_tfrec)
                   .map(partial(parse_test_fun, params = params), num_parallel_calls=AUTOTUNE)
                   .map(partial(convert_to_spect, params = params), num_parallel_calls=AUTOTUNE)
                   .map(partial(fit_output, params = params), num_parallel_calls=AUTOTUNE))

    model = FetchModel(params)

    recids = []
    rows = []
    for data,recid in tqdm(testdataset):
        prob = model(data, training = False)
        prob = tf.reduce_max(prob, axis = 0)
        row = [recid[0].numpy().decode()] + list(prob.numpy())
        rows.append(row)
    cols = ['recording_id'] + ['s{}'.format(i) for i in range(24)]
    subdf = pd.DataFrame(rows, columns = cols)
    subdf.to_csv(params.subm_file,index=False)

# STEP 1

In [ ]:
print(GPARAMS)

In [ ]:
testdataset = (tfd.TFRecordDataset(GPARAMS.all_test_tfrec)
               .map(partial(parse_test_fun, params = GPARAMS), num_parallel_calls=AUTOTUNE)
               .map(partial(convert_to_spect, params = GPARAMS), num_parallel_calls=AUTOTUNE)
               .map(partial(fit_output, params = GPARAMS), num_parallel_calls=AUTOTUNE))

model = FetchModel(GPARAMS)

In [ ]:
recids = []
rows = []
for data,recid in tqdm(testdataset):
    prob = model(data, training = False)
    prob = tf.reduce_max(prob, axis = 0)
    row = [recid[0].numpy().decode()] + list(prob.numpy())
    rows.append(row)
cols = ['recording_id'] + ['s{}'.format(i) for i in range(24)]
subdf = pd.DataFrame(rows, columns = cols)
subdf.to_csv(GPARAMS.subm_file,index=False)

# STEP 2

In [ ]:
path_tp_model = '/kaggle/input/model-weights-part4/modelv3_e28_i64_tp_s512_s2048.h5'
subm_file = "submission_v3_e28_i64_tp_s512_s2048_f20000.csv"
mel_frq_h = 20000.0

GPARAMS = Params(path_tp_model = path_tp_model, subm_file = subm_file, mel_frq_h = mel_frq_h)

In [ ]:
print(GPARAMS)

In [ ]:
testdataset = (tfd.TFRecordDataset(GPARAMS.all_test_tfrec)
               .map(partial(parse_test_fun, params = GPARAMS), num_parallel_calls=AUTOTUNE)
               .map(partial(convert_to_spect, params = GPARAMS), num_parallel_calls=AUTOTUNE)
               .map(partial(fit_output, params = GPARAMS), num_parallel_calls=AUTOTUNE))

model = FetchModel(GPARAMS)

In [ ]:
recids = []
rows = []
for data,recid in tqdm(testdataset):
    prob = model(data, training = False)
    prob = tf.reduce_max(prob, axis = 0)
    row = [recid[0].numpy().decode()] + list(prob.numpy())
    rows.append(row)
cols = ['recording_id'] + ['s{}'.format(i) for i in range(24)]
subdf = pd.DataFrame(rows, columns = cols)
subdf.to_csv(GPARAMS.subm_file,index=False)

# STEP 3

In [ ]:
path_tp_model = '/kaggle/input/model-weights-part4/modelv3_e34_i64_tp_s512_s2048.h5'
subm_file = "submission_v3_e34_i64_tp_s512_s2048_f16000.csv"
mel_frq_h = 16000.0

GPARAMS = Params(path_tp_model = path_tp_model, subm_file = subm_file, mel_frq_h = mel_frq_h)

In [ ]:
print(GPARAMS)

In [ ]:
testdataset = (tfd.TFRecordDataset(GPARAMS.all_test_tfrec)
               .map(partial(parse_test_fun, params = GPARAMS), num_parallel_calls=AUTOTUNE)
               .map(partial(convert_to_spect, params = GPARAMS), num_parallel_calls=AUTOTUNE)
               .map(partial(fit_output, params = GPARAMS), num_parallel_calls=AUTOTUNE))

model = FetchModel(GPARAMS)

In [ ]:
recids = []
rows = []
for data,recid in tqdm(testdataset):
    prob = model(data, training = False)
    prob = tf.reduce_max(prob, axis = 0)
    row = [recid[0].numpy().decode()] + list(prob.numpy())
    rows.append(row)
cols = ['recording_id'] + ['s{}'.format(i) for i in range(24)]
subdf = pd.DataFrame(rows, columns = cols)
subdf.to_csv(GPARAMS.subm_file,index=False)

# STEP 4

In [ ]:
path_tp_model = '/kaggle/input/model-weights-part4/modelv3_e34_i64_tp_s512_s2048.h5'
subm_file = "submission_v3_e34_i64_tp_s512_s2048_f18000.csv"
mel_frq_h = 18000.0

GPARAMS = Params(path_tp_model = path_tp_model, subm_file = subm_file, mel_frq_h = mel_frq_h)

In [ ]:
print(GPARAMS)

In [ ]:
testdataset = (tfd.TFRecordDataset(GPARAMS.all_test_tfrec)
               .map(partial(parse_test_fun, params = GPARAMS), num_parallel_calls=AUTOTUNE)
               .map(partial(convert_to_spect, params = GPARAMS), num_parallel_calls=AUTOTUNE)
               .map(partial(fit_output, params = GPARAMS), num_parallel_calls=AUTOTUNE))

model = FetchModel(GPARAMS)

In [ ]:
recids = []
rows = []
for data,recid in tqdm(testdataset):
    prob = model(data, training = False)
    prob = tf.reduce_max(prob, axis = 0)
    row = [recid[0].numpy().decode()] + list(prob.numpy())
    rows.append(row)
cols = ['recording_id'] + ['s{}'.format(i) for i in range(24)]
subdf = pd.DataFrame(rows, columns = cols)
subdf.to_csv(GPARAMS.subm_file,index=False)

# STEP 5

In [ ]:
path_tp_model = '/kaggle/input/model-weights-part4/modelv3_e34_i64_tp_s512_s2048.h5'
subm_file = "submission_v3_e34_i64_tp_s512_s2048_f20000.csv"
mel_frq_h = 20000.0

GPARAMS = Params(path_tp_model = path_tp_model, subm_file = subm_file, mel_frq_h = mel_frq_h)

In [ ]:
print(GPARAMS)

In [ ]:
testdataset = (tfd.TFRecordDataset(GPARAMS.all_test_tfrec)
               .map(partial(parse_test_fun, params = GPARAMS), num_parallel_calls=AUTOTUNE)
               .map(partial(convert_to_spect, params = GPARAMS), num_parallel_calls=AUTOTUNE)
               .map(partial(fit_output, params = GPARAMS), num_parallel_calls=AUTOTUNE))

model = FetchModel(GPARAMS)

In [ ]:
recids = []
rows = []
for data,recid in tqdm(testdataset):
    prob = model(data, training = False)
    prob = tf.reduce_max(prob, axis = 0)
    row = [recid[0].numpy().decode()] + list(prob.numpy())
    rows.append(row)
cols = ['recording_id'] + ['s{}'.format(i) for i in range(24)]
subdf = pd.DataFrame(rows, columns = cols)
subdf.to_csv(GPARAMS.subm_file,index=False)

In [ ]:
for i in range(100000):
    print("########################################################################################################")